pip install torch transformers sklearn pandas

For Mac M1:

curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [1]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm Elon Musk,", max_length=30, num_return_sequences=5)

/opt/anaconda3/envs/DL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm Elon Musk, I'm CEO of Intel. In my previous years working at Intel, I spent many years on the research and development"},
 {'generated_text': "Hello, I'm Elon Musk, and I'm President/CEO of Tesla. And it's so humbling. This is the kind of kind of"},
 {'generated_text': "Hello, I'm Elon Musk, the world's most valuable entrepreneur, and I wanted to hear from you about why I'm passionate about building a network"},
 {'generated_text': "Hello, I'm Elon Musk, a billionaire tech investor and investor in SpaceX and the Space Exploration Technologies Corporation. If you aren't familiar he is Tesla"},
 {'generated_text': "Hello, I'm Elon Musk, the CEO and Co-Founder of SpaceX and founder of the SpaceX Education Lab. It's not easy, but"}]

The next step is to use all tweets to build a TextDataset. The TextDataset is a custom implementation of the Pytroch Dataset class implemented by the transformers library. 

First, we split the tweets into a train and test section then write them into a train_dataset.txt and test_dataset.txt



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("dataset/train_cleaned.csv")['content'].to_numpy()
train, test = train_test_split(data,test_size=0.15)
traindata = ''
testdata = ''
for i in train:
    traindata += i.replace("&amp", "") +'\n'
f = open('train_dataset.txt','w')
f.write(traindata)
for i in test:
    testdata += i.replace("&amp","") +'\n'
f = open('test_dataset.txt','w')
f.write(testdata)

The next step is to download the tokenizer. We use the tokenizer from the german-gpt2 model.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling,LineByLineTextDataset

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = LineByLineTextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = LineByLineTextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)


In [ ]:
from transformers import LineByLineTextDataset
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2-musk", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #prediction_loss_only=True,
)

In [ ]:
for id, tensor  in enumerate(train_dataset):
    print(tensor)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

tweet = pipeline('text-generation',model='gpt2-musk', tokenizer=tokenizer )

In [ ]:
#generator = pipeline('text-generation', model='gpt2')
from transformers import pipeline, set_seed
set_seed(42)
tweet("With steel membrane wings like a Dragon,", max_length=50, num_return_sequences=5)